In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

/home/surya/DL/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.
ERROR (theano.gpuarray): pygpu was configured but could not be imported or is too old (version 0.6 or higher required)
NoneType


In [ ]:
from keras.preprocessing.sequence import 

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample = pd.read_csv('data/sample_submission.csv')

In [3]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [5]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


In [70]:
max_len = max([len(i.split()) for i in train.text.values])
max_len

861

In [3]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [4]:
lbl_encoder = preprocessing.LabelEncoder()
y = lbl_encoder.fit_transform(train.author.values)

In [5]:
x_train,x_valid,y_train,y_valid = train_test_split(train.text.values,y,
                                                       test_size=0.5,shuffle=True)

In [6]:
x_train.shape,x_valid.shape,y_train.shape,y_valid.shape

((9789,), (9790,), (9789,), (9790,))

In [11]:
x_train

array([ 'In the morning, I procured, at the Prefecture, a full report of all the evidence elicited, and, at the various newspaper offices, a copy of every paper in which, from first to last, had been published any decisive information in regard to this sad affair.',
       'And I saw the world battling against blackness; against the waves of destruction from ultimate space; whirling, churning; struggling around the dimming, cooling sun.',
       "Among the other transcendant attributes of Mozart's music, it possesses more than any other that of appearing to come from the heart; you enter into the passions expressed by him, and are transported with grief, joy, anger, or confusion, as he, our soul's master, chooses to inspire.",
       ...,
       'At certain stages of the ritual they did grovelling obeisance, especially when he held above his head that abhorrent Necronomicon he had taken with him; and I shared all the obeisances because I had been summoned to this festival by the writin

## fitting the data with classical machine learning algorithms

In [12]:
tfv = TfidfVectorizer(analyzer='word',min_df=3,stop_words='english',
                      use_idf=1,sublinear_tf=1,smooth_idf=1)
tfv.fit(list(x_train)+list(x_valid))
x_train_tfv = tfv.transform(x_train)
x_valid_tfv = tfv.transform(x_valid)

In [13]:
clf = LogisticRegression(C=1.0)
clf.fit(x_train_tfv,y_train)
predictions = clf.predict_proba(x_valid_tfv)

print("log loss is: ", multiclass_logloss(y_valid,predictions))

log loss is:  0.701744292595


In [14]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,3),
                      stop_words='english')
ctv.fit(list(x_train)+list(x_valid))
x_train_ctv = ctv.transform(x_train)
x_valid_ctv = ctv.transform(x_valid)

In [15]:
clf = LogisticRegression(C=1.0)
clf.fit(x_train_ctv,y_train)
predictions = clf.predict_proba(x_valid_ctv)

print("log loss using countVectorizer is: ",multiclass_logloss(y_valid,predictions) )

log loss using countVectorizer is:  0.593498356688


### naive bayes

In [16]:
#on TFIDF
nvb = MultinomialNB()
nvb.fit(x_train_tfv,y_train)
predictions = nvb.predict_proba(x_valid_tfv)

print("log loss using naive bayes for TFIDF: ",multiclass_logloss(y_valid,predictions))

log loss using naive bayes for TFIDF:  0.654915406219


In [17]:
#on CountVectors
nvb = MultinomialNB()
nvb.fit(x_train_ctv,y_train)
predictions = nvb.predict_proba(x_valid_ctv)

print("log loss using naive bayes for countvectors: ",multiclass_logloss(y_valid,predictions))

log loss using naive bayes for countvectors:  0.496418738124


it seems naive bayes with count vectors performed great.

### SVM

In [18]:
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(x_train_tfv)
x_train_tfv_svd = svd.transform(x_train_tfv)
x_valid_tfv_svd = svd.transform(x_valid_tfv)

In [19]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train_tfv_svd)
x_train_svd = scaler.transform(x_train_tfv_svd)
x_valid_svd = scaler.transform(x_valid_tfv_svd)

In [31]:
clf = SVC(C=1.0,probability=True)
clf.fit(x_train_svd,y_train)
preds = clf.predict_proba(x_valid_svd)

print("log loss of the SVM is: ",multiclass_logloss(y_valid,preds))

log loss of the SVM is:  0.7555068647


### word vectors using spacy

In [39]:
from spacy.vectors import Vectors
import spacy

nlp =  spacy.load('en')
#nlp.vocab.vectors.from_glove('data/glove.840B.300d.txt')

In [42]:
xtrain_token=[]
for i in x_train:
    doc = nlp(i)
    xtrain_token.append([token.text for token in doc ])


In [44]:
xvalid_token=[]
for i in x_valid:
    doc = nlp(i)
    xvalid_token.append([token.text for token in doc ])


In [45]:
xtrain_token
xvalid_token

[['It',
  'is',
  'true',
  ',',
  'the',
  'populace',
  'retained',
  'themselves',
  ';',
  'but',
  'there',
  'arose',
  'a',
  'perpetual',
  'hum',
  'and',
  'bustle',
  'from',
  'the',
  'throng',
  'round',
  'the',
  'palace',
  ',',
  'which',
  'added',
  'to',
  'the',
  'noise',
  'of',
  'fireworks',
  ',',
  'the',
  'frequent',
  'explosion',
  'of',
  'arms',
  ',',
  'the',
  'tramp',
  'to',
  'and',
  'fro',
  'of',
  'horsemen',
  'and',
  'carriages',
  ',',
  'to',
  'which',
  'effervescence',
  'he',
  'was',
  'the',
  'focus',
  ',',
  'retarded',
  'his',
  'recovery',
  '.'],
 ['In',
  'the',
  'meantime',
  ',',
  'however',
  ',',
  'I',
  'had',
  'no',
  'notion',
  'of',
  'being',
  'thwarted',
  'touching',
  'the',
  'information',
  'I',
  'desired',
  '.'],
 ['Not',
  'for',
  'a',
  'moment',
  'did',
  'I',
  'believe',
  'that',
  'the',
  'tale',
  'had',
  'any',
  'really',
  'substantial',
  'foundation',
  ';',
  'but',
  'none',
  'the

### WordVectors

In [7]:
loc = "data/"
import bcolz
import pickle
import sys   


In [8]:
xtrain = []
xtrain = np.array([i.split() for i in x_train])
xtrain
xvalid = np.array([i.split() for i in x_valid])
xvalid

array([ list(['It', 'is', 'true,', 'the', 'populace', 'retained', 'themselves;', 'but', 'there', 'arose', 'a', 'perpetual', 'hum', 'and', 'bustle', 'from', 'the', 'throng', 'round', 'the', 'palace,', 'which', 'added', 'to', 'the', 'noise', 'of', 'fireworks,', 'the', 'frequent', 'explosion', 'of', 'arms,', 'the', 'tramp', 'to', 'and', 'fro', 'of', 'horsemen', 'and', 'carriages,', 'to', 'which', 'effervescence', 'he', 'was', 'the', 'focus,', 'retarded', 'his', 'recovery.']),
       list(['In', 'the', 'meantime,', 'however,', 'I', 'had', 'no', 'notion', 'of', 'being', 'thwarted', 'touching', 'the', 'information', 'I', 'desired.']),
       list(['Not', 'for', 'a', 'moment', 'did', 'I', 'believe', 'that', 'the', 'tale', 'had', 'any', 'really', 'substantial', 'foundation;', 'but', 'none', 'the', 'less', 'the', 'account', 'held', 'a', 'hint', 'of', 'genuine', 'terror,', 'if', 'only', 'because', 'it', 'brought', 'in', 'references', 'to', 'strange', 'jewels', 'clearly', 'akin', 'to', 'the', 'ma

In [48]:
import pdb
import os
from tensorflow.contrib.learn.python import preprocessing

In [10]:
def load_array(fname):
    return bcolz.open(fname)[:]

In [11]:
vecs = load_array('data/6B.50d.dat')
words = pickle.load(open('data/6B.50d_words.pkl','rb'),encoding='utf8'),
idx = pickle.load(open('data/6B.50d_idx.pkl','rb'),encoding='utf8')

In [24]:
xtrain_fltr = [[i.lower().strip(',.;"') for i in s] for s in xtrain]
xvalid_fltr = [[i.lower().strip(',.";') for i in s] for s in xvalid]

In [ ]:
uniquewords = []
def fetchUnique(li):
    for i in li:
        if not i in uniquewords:
            uniquewords.append(i)
    return uniquewords
    

In [73]:
p = preprocessing.text.VocabularyProcessor(max_document_length=500)
xtrain_ids = np.array(list(p.fit_transform(x_train)))
xvalid_ids = np.array(list(p.fit_transform(x_valid)))

In [74]:
xtrain_ids.shape,xvalid_ids.shape

((9789, 500), (9790, 500))

In [102]:
xtrain_ids_v[np.where(xtrain_ids_v > 21000)].size

0

In [101]:
vocab_size = 21000
xtrain_ids_v = np.array([[i if i<vocab_size else vocab_size-1 for i in s] for s in xtrain_ids])
xvalid_ids_v = np.array([[i if i<vocab_size else vocab_size-1 for i in s] for s in xvalid_ids])


In [11]:
#fi = open("/home/surya/Documents/dataset/kaggle/predict_author/dict.txt","a+")

In [ ]:
# load the GloVe vectors in a dictionary:
indx_len = 0
embeddings_index = {}
f = open('data/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
    except:
        print(values[1:])
    embeddings_index[word] =coefs
    fi.write(str(embeddings_index))
    embeddings_index = {}
    indx_len +=1
    #pdb.set_trace()
f.close()
fi.close()
print('Found %s word vectors.' % indx_len)

In [104]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(idx[w])
        except:
            continue
    M = np.array(M)
    return M

# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in x_train]
xvalid_glove = [sent2vec(x) for x in x_valid]
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)


In [104]:
seq_len = 500
#trn_seq = sequence.pad_sequences(xtrain_glove,maxlen=seq_len,value=0.0)
#tst_seq = sequence.pad_sequences(xvalid_glove,maxlen=seq_len,value=0.0)

In [111]:
trn_seq[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [57]:
vecs = np.array(vecs)

In [ ]:
idx2word = {}

In [52]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = idx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [60]:
emb = create_emb()

NameError: name 'idx2word' is not defined

In [105]:
vocab_size = 21000
seq_len = 500

In [108]:
ytrain_enc

array([[ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.]])

## LSTM Network for classifying the Author

In [107]:

# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(y_train)
yvalid_enc = np_utils.to_categorical(y_valid)

In [121]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(x_train)
xvalid_seq = token.texts_to_sequences(x_valid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [109]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(vocab_size,
                     300,
                     input_length=seq_len,
                     trainable=True))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_ids_v, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_ids_v, yvalid_enc), callbacks=[earlystop])

<<!! BUG IN FGRAPH.REPLACE OR A LISTENER !!>> <class 'TypeError'> ('The type of the replacement must be compatible with the type of the original Variable.', mrg_uniform{TensorType(float32, 3D),no_inplace}.1, mrg_uniform{TensorType(float32, 3D),inplace}.1, TensorType(float32, (False, True, False)), TensorType(float32, 3D), 'mrg_random_make_inplace') mrg_random_make_inplace
ERROR (theano.gof.opt): Optimization failure due to: mrg_random_make_inplace
ERROR:theano.gof.opt:Optimization failure due to: mrg_random_make_inplace
ERROR (theano.gof.opt): node: mrg_uniform{TensorType(float32, 3D),no_inplace}(<TensorType(int32, matrix)>, <TensorType(int64, vector)>)
ERROR:theano.gof.opt:node: mrg_uniform{TensorType(float32, 3D),no_inplace}(<TensorType(int32, matrix)>, <TensorType(int64, vector)>)
ERROR (theano.gof.opt): TRACEBACK:
ERROR:theano.gof.opt:TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/surya/DL/lib/python3.5/site-packages/theano/gof/opt.py", line 202

ERROR:theano.gof.opt:Optimization failure due to: mrg_random_make_inplace
ERROR (theano.gof.opt): node: mrg_uniform{TensorType(float32, 3D),no_inplace}(<TensorType(int32, matrix)>, <TensorType(int64, vector)>)
ERROR:theano.gof.opt:node: mrg_uniform{TensorType(float32, 3D),no_inplace}(<TensorType(int32, matrix)>, <TensorType(int64, vector)>)
ERROR (theano.gof.opt): TRACEBACK:
ERROR:theano.gof.opt:TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/surya/DL/lib/python3.5/site-packages/theano/gof/opt.py", line 2022, in process_node
    remove=remove)
  File "/home/surya/DL/lib/python3.5/site-packages/theano/gof/toolbox.py", line 391, in replace_all_validate_remove
    chk = fgraph.replace_all_validate(replacements, reason)
  File "/home/surya/DL/lib/python3.5/site-packages/theano/gof/toolbox.py", line 340, in replace_all_validate
    fgraph.replace(r, new_r, reason=reason, verbose=False)
  File "/home/surya/DL/lib/python3.5/site-packages/theano/gof/fg.py", 

Train on 9789 samples, validate on 9790 samples
Epoch 1/100


KeyboardInterrupt: 